<h1><center><font size="6">Transfer Learning with Keras for Cifar-10</font></center></h1>

# Introduction


## Dataset

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.


# Load packages

In [ ]:
# Setup one GPU for tensorflow
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# The GPU id to use, "0", "1", etc.
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

# https://keras.io/applications/#documentation-for-individual-models
from keras.applications.resnet50 import ResNet50
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import cv2
import numpy as np
import tensorflow as tf

# Limit tensorflow gpu usage
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
sess = tf.Session(config=config)


## Parameters

In [ ]:
# Some constants
IMG_ROWS = 32
IMG_COLS = 32
NUM_CLASSES = 10
TEST_SIZE = 0.2
RANDOM_STATE = 2018

N_TRAIN_SAMPLES = 5000
N_TEST_SAMPLES = 1000

#Model
NO_EPOCHS = 50
BATCH_SIZE = 64
NET_IMG_ROWS = 224
NET_IMG_COLS = 224

# Load data

In [ ]:
(x_train_data, y_train_data), (x_test_data, y_test_data) = cifar10.load_data()

# Create a dictionary for each type of label 
LABELS = {
    0: "airplane",
    1: "automobile",
    2: "bird",
    3: "cat",
    4: "deer",
    5: "dog",
    6: "frog",
    7: "horse",
    8: "ship",
    9: "truck"
}


In [ ]:
# For this tutorial we will work on a smaller subset
train_idxs = np.random.choice(np.arange(x_train_data.shape[0]), size=N_TRAIN_SAMPLES, replace=False)
test_idxs = np.random.choice(np.arange(x_test_data.shape[0]), size=N_TEST_SAMPLES, replace=False)

x_train_data = x_train_data[train_idxs, :, :, :]
y_train_data = y_train_data[train_idxs]
x_test_data = x_test_data[test_idxs, :, :, :]
y_test_data = y_test_data[test_idxs]


In [ ]:
print("Cifar-10 x_train shape: {}".format(x_train_data.shape))
print("Cifar-10 y_train shape: {}".format(y_train_data.shape))
print("Cifar-10 x_test shape: {}".format(x_test_data.shape))
print("Cifar-10 y_test shape: {}".format(y_test_data.shape))

## Data preprocessing

In [ ]:
def data_preprocessing(x_data, y_data):
    out_x = np.array([cv2.resize(img, (NET_IMG_COLS, NET_IMG_ROWS)).astype(np.float32)
                      for img in x_data])
    out_y = to_categorical(y_data, len(np.unique(y_data)))
    
    return out_x, out_y

# prepare the data
X, y = data_preprocessing(x_train_data, y_train_data)
X_test, y_test = data_preprocessing(x_test_data, y_test_data)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)


# Build model

In [ ]:
# Create the base model with weights pre-trained on ImageNet.
# We will use the Resnet50 model
# - [Deep Residual Learning for Image Recognition](
#    https://arxiv.org/abs/1512.03385)
base_model = ResNet50(input_shape=(NET_IMG_ROWS, NET_IMG_COLS, 3),  # Input image size
                      weights='imagenet',                           # Use imagenet pre-trained weights
                      include_top=False,                            # Drop classification layer
                      pooling='avg')                                # Global AVG pooling for the 
                                                                    #  output feature vector


### Inspect the model

Let's check the model we initialized.

In [ ]:
base_model.summary()

### Add the classification layer

In [ ]:
# get the output feature vector from the base model
x = base_model.output
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Check the final model
model.summary()

# Training

In [ ]:
# Training data generator
datagen_train = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    )

# Validation data generator
datagen_val = ImageDataGenerator(
    rescale=1./255)


In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE)


In [ ]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(model.layers):
    print(i, layer.name)


In [ ]:
# We will finetune from add_9
for layer in model.layers[:99]:
    layer.trainable = False
for layer in model.layers[99:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train!
train_model = model.fit_generator(
    datagen_train.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=NO_EPOCHS,
    validation_data=datagen_val.flow(X_val, y_val, batch_size=BATCH_SIZE),
    steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
    validation_steps=X_val.shape[0] // BATCH_SIZE)
